In [2]:
import getpass
import socket
import requests
import json
import zipfile
import os
import io
import s3fs

In [7]:
with open('.cred') as json_file:  
    data = json.load(json_file)

In [8]:
# Request token from Common Metadata Repository using Earthdata credentials
token_api_url = 'https://api.echo.nasa.gov/echo-rest/tokens'
hostname = socket.gethostname()
ip = socket.gethostbyname(hostname)

data = {
    'token': {
        'username': data['uid'],
        'password': data['pwd'],
        'client_id': 'NSIDC_client_id',
        'user_ip_address': ip
    }
}
headers={'Accept': 'application/json'}
response = requests.post(token_api_url, json=data, headers=headers)
token = json.loads(response.content)['token']['id']

In [14]:
response = requests.get("https://n5eil02u.ecs.nsidc.org/egi/request?short_name=ATL03&version=205&temporal=2018-10-17T00:00:00,2018-10-17T23:59:59&bounding_box=-115,35,-99,42&agent=NO&token={}&page_size=1&email=arendta@uw.edu".format(token))              

In [15]:
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall(os.getcwd())

In [ ]:
# Connect to bucket
import s3fs
bucket = 'pangeo-data-upload-oregon'
fs = s3fs.S3FileSystem()
# List contents
dataDir = 'pangeo-data-upload-oregon/icesat2/atl03'
fs.ls(dataDir)

In [ ]:
# Download an HDF5 to home directory
filename = 'ATL06_20181019051247_03140110_203_01.h5'
data = dataDir + '/' + filename
fs.get(data, filename)

In [7]:
import xarray as xr

In [10]:
ds = xr.open_dataset('ATL03_20181017101822_02870106_205_01.h5')

In [11]:
ds

<xarray.Dataset>
Dimensions:       (ds_surf_type: 5, ds_xyz: 3)
Coordinates:
  * ds_surf_type  (ds_surf_type) int32 1 2 3 4 5
  * ds_xyz        (ds_xyz) int32 1 2 3
Data variables:
    *empty*
Attributes:
    granule_type:                       ATL03
    short_name:                         ATL03
    level:                              L2
    description:                        Photon heights determined by ATBD Alg...
    contributor_name:                   Thomas E Neumann (thomas.neumann@nasa...
    contributor_role:                   Instrument Engineer, Investigator, Pr...
    Conventions:                        CF-1.6
    date_type:                          UTC
    featureType:                        trajectory
    geospatial_lat_units:               degrees_north
    geospatial_lon_units:               degrees_east
    identifier_product_doi:             10.5067/ATLAS/ATL03.001
    identifier_product_doi_authority:   http://dx.doi.org
    identifier_product_type:            ATL03
